In [ ]:
import pandas as pd
import os
import os.path as op
import glob
import re
import nibabel as nib
import numpy as np

from nilearn.image import resample_to_img
import matplotlib.pyplot as plt

In [ ]:
def dice_coefficient(mask1, mask2):
    """
    Calculate Dice similarity coefficient between two binary masks.
    
    Dice = 2 * |X ∩ Y| / (|X| + |Y|)
    
    Parameters:
    -----------
    mask1, mask2 : array-like or nibabel image
        Binary masks (3D arrays or nibabel images)
    
    Returns:
    --------
    dice : float
        Dice coefficient (0-1, where 1 = perfect overlap)
    """
    # Convert to arrays if nibabel images
    if hasattr(mask1, 'get_fdata'):
        mask1 = mask1.get_fdata()
    if hasattr(mask2, 'get_fdata'):
        mask2 = mask2.get_fdata()
    
    # Flatten and binarize
    mask1_flat = (mask1.flatten() > 0).astype(int)
    mask2_flat = (mask2.flatten() > 0).astype(int)
    
    # Calculate intersection and sizes
    intersection = np.sum(mask1_flat * mask2_flat)
    size1 = np.sum(mask1_flat)
    size2 = np.sum(mask2_flat)
    
    # Avoid division by zero
    if size1 + size2 == 0:
        return 0.0
    
    dice = (2.0 * intersection) / (size1 + size2)
    return dice

In [ ]:
def pairwise_dice_scores(roi_files):
    """
    Calculate pairwise Dice coefficients between all subject-specific ROIs.
    
    Parameters:
    -----------
    roi_files : list
        List of file paths to subject-specific ROI NIfTI files
    
    Returns:
    --------
    dice_matrix : numpy.ndarray
        Symmetric matrix where dice_matrix[i,j] is the Dice coefficient 
        between ROI i and ROI j
    roi_names : list
        List of ROI filenames for indexing the matrix
    pairwise_dice_values : numpy.ndarray
        1D array of unique pairwise Dice values (excluding diagonal)
    """
    n_rois = len(roi_files)
    dice_matrix = np.zeros((n_rois, n_rois))
    roi_names = [os.path.basename(f) for f in roi_files]
    
    print(f"Calculating pairwise Dice scores for {n_rois} ROIs...")
    print(f"Total comparisons: {n_rois * (n_rois - 1) // 2}")
    
    # Calculate pairwise dice coefficients
    for i in range(n_rois):
        roi1 = nib.load(roi_files[i])
        for j in range(i, n_rois):  # Only upper triangle + diagonal
            if i == j:
                dice_matrix[i, j] = 1.0  # Perfect self-similarity
            else:
                roi2 = nib.load(roi_files[j])
                dice_score = dice_coefficient(roi1, roi2)
                dice_matrix[i, j] = dice_score
                dice_matrix[j, i] = dice_score  # Symmetric matrix
        
        if (i + 1) % 100 == 0:  # Progress update every 100 ROIs
            print(f"Processed {i + 1}/{n_rois} ROIs")
    
    # Extract unique pairwise values (upper triangle, excluding diagonal)
    pairwise_dice_values = dice_matrix[np.triu_indices(n_rois, k=1)]
    
    print("Pairwise Dice calculation completed!")
    print(f"\nSummary Statistics:")
    print(f"Mean Dice: {np.mean(pairwise_dice_values):.3f}")
    print(f"SD Dice: {np.std(pairwise_dice_values):.3f}")
    print(f"Median Dice: {np.median(pairwise_dice_values):.3f}")
    print(f"Range: [{np.min(pairwise_dice_values):.3f}, {np.max(pairwise_dice_values):.3f}]")
    
    return dice_matrix, roi_names, pairwise_dice_values

In [ ]:
rois_dir = "./dset/seed-regions"

In [ ]:
# Find all subject-specific habenula ROI files
subj_rois_dir = op.join(rois_dir, "subj-spec-hbs")
subj_roi_files = glob.glob(op.join(subj_rois_dir, "*.nii.gz"))

print(subj_roi_files)

In [ ]:
# Calculate pairwise Dice scores between all subject-specific ROIs
dice_matrix, roi_names, pairwise_values = pairwise_dice_scores(subj_roi_files)

In [ ]:
# Create histogram of pairwise Dice scores


plt.figure(figsize=(10, 6))
plt.hist(pairwise_values, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Dice Coefficient', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Pairwise Dice Scores Across Individual Hand-Drawn ROIs', fontsize=14)
plt.axvline(np.mean(pairwise_values), color='red', linestyle='--', linewidth=2, 
            label=f'Mean = {np.mean(pairwise_values):.3f}')
plt.axvline(np.median(pairwise_values), color='blue', linestyle='--', linewidth=2,
            label=f'Median = {np.median(pairwise_values):.3f}')
plt.legend(fontsize=11)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Ensure all NIfTI files are loaded, binarized, and saved in AFNI-compatible format
afni_nii_files = []
for i, roi_file in enumerate(subj_roi_files):
    img = nib.load(roi_file)
    data = img.get_fdata()
    # Binarize: set all nonzero voxels to 1
    bin_data = (data > 0).astype(np.uint8)
    bin_img = nib.Nifti1Image(bin_data, img.affine, img.header)
    temp_fn = os.path.join(subj_rois_dir, f"afni_bin_roi_{i}.nii.gz")
    nib.save(bin_img, temp_fn)
    afni_nii_files.append(temp_fn)

# Use AFNI 3dMean to average all binarized ROIs
avg_subj_roi = os.path.join(subj_rois_dir, "subj_averaged_hbroi.nii.gz")
mean_cmd = f"3dMean -prefix {avg_subj_roi} {' '.join(afni_nii_files)}"
os.system(mean_cmd)

print(f"Averaged binarized ROI saved to: {avg_subj_roi}")
# Clean up temporary binarized ROI files
print("Cleaning up temporary binarized ROI files...")
for temp_file in afni_nii_files:
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"Deleted: {temp_file}")
print("Cleanup completed.")

In [ ]:
# load the atlas based roi
atlas_roi_dir = op.join(rois_dir, "atlas-based-hb")
atlas_roi= glob.glob(op.join(atlas_roi_dir, "*dilated.nii.gz"))

print(atlas_roi)

In [ ]:

atlas_roi_img = nib.load(atlas_roi[0])  # Take first file from the list
avg_roi_img = nib.load(avg_subj_roi)

atlas_roi_resampled = resample_to_img(atlas_roi_img, avg_roi_img, interpolation='nearest')
print(f"Atlas ROI shape: {atlas_roi_resampled.shape}")


In [ ]:
# Calculate Dice coefficient between subject-averaged ROI and atlas ROI
atlas_vs_subj_dice = dice_coefficient(atlas_roi_resampled, avg_roi_img)

print(f"Dice coefficient between Atlas ROI and Subject-Averaged ROI: {atlas_vs_subj_dice:.4f}")